In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from tensorflow.keras.models import Model
import tensorflow_addons as tfa

# Sample data
sentences = [['I', 'love', 'this', 'phone'], ['This', 'movie', 'is', 'terrible']]
sentence_labels = [['O', 'B-POS', 'I-POS', 'I-POS'], ['O', 'B-NEG', 'I-NEG', 'I-NEG']]

# Create word and tag dictionaries for mapping
words = list(set(word for sentence in sentences for word in sentence))
words.append("ENDPAD")
n_words = len(words)
tags = list(set(tag for tag_seq in sentence_labels for tag in tag_seq))
n_tags = len(tags)

word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: w for w, i in tag2idx.items()}

# Convert each sentence into a sequence of numbers and pad sequences
maxlen = max(len(s) for s in sentences)
X_data = [[word2idx[w] for w in s] for s in sentences]
X_data = pad_sequences(X_data, maxlen=maxlen, padding='post')
Y_data = [[tag2idx[w] for w in s] for s in sentence_labels]
Y_data = pad_sequences(Y_data, maxlen=maxlen, padding='post')

# Build the model
input = Input(shape=(maxlen,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=maxlen)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(50, activation="relu"))(model)
model = TimeDistributed(Dense(n_tags))(model)

crf = tfa.layers.CRF(n_tags)
out = crf(model)

model = Model(input, out)
model.compile(optimizer="adam", loss=crf.get_loss, metrics=[crf.get_accuracy])
model.summary()

# Train the model
model.fit(X_data, Y_data, batch_size=2, epochs=5, validation_split=0.1, verbose=1)


c:\Users\Ephyrus\anaconda3\envs\number4\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


AttributeError: 'CRF' object has no attribute 'get_loss'